In [2]:
import os
from bs4 import BeautifulSoup as bs
import requests
from splinter import Browser
import pandas as pd
import pprint
import datetime as dt

In [3]:
url_corona = "https://www.worldometers.info/coronavirus/#countries"
html_corona = requests.get(url_corona)
html_corona = html_corona.text
#html_corona

In [4]:
soup_corona = bs(html_corona, 'html.parser')

In [5]:
corona_table = soup_corona.find(class_="table-bordered")

corona_table = str(corona_table)


In [6]:
corona_df = pd.read_html(corona_table)[0]

#replacing blanks with 0
corona_df.fillna(0,inplace = True)

#removing the total row)
corona_df = corona_df[~corona_df['Country,Other'].isin(['Total:', 'Total', 'total'])]
corona_df.to_csv('new_data.csv')
corona_df

,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop
0,China,80783,29.0,3158.0,22.0,61493.0,16132,4492.0,56.1
1,Italy,10149,0.0,631.0,0.0,1004.0,8514,877.0,167.9
2,Iran,8042,0.0,291.0,0.0,2731.0,5020,0.0,95.7
3,S. Korea,7755,242.0,61.0,1.0,288.0,7406,54.0,151.3
4,France,1784,0.0,33.0,0.0,12.0,1739,86.0,27.3
...,...,...,...,...,...,...,...,...,...
114,Liechtenstein,1,0.0,0.0,0.0,0.0,1,0.0,0.0
115,Mongolia,1,0.0,0.0,0.0,0.0,1,0.0,0.3
116,St. Barth,1,0.0,0.0,0.0,0.0,1,0.0,0.0
117,Togo,1,0.0,0.0,0.0,0.0,1,0.0,0.1


In [23]:
# storing the rate of growth for analysis

#importing previous data
growth_last = pd.read_csv("growth.csv")

#staging new data to add
growth_df = pd.DataFrame()


growth_df["Date"]=[0]
growth_df["NewCases"]=[0]
growth_df["TotalCases"] =[0]
growth_df["VarianceMultiple"]=[0]
growth_df["GrowthFactor"]=[0]

growth_df["Date"]= dt.datetime.today()
growth_df["NewCases"] = corona_df["NewCases"].sum()
growth_df["TotalCases"] =corona_df["TotalCases"].sum()
growth_df["VarianceMultiple"]= 1 + (growth_df["NewCases"]/growth_df["TotalCases"])
growth_df["GrowthFactor"]=[growth_df["NewCases"][0]/growth_last["NewCases"][len(growth_last)-1]]

#appending new data 
updated_growth_df = growth_last.append(growth_df)
updated_growth_df.reset_index(inplace=True,drop=True)
# #updated_growth_df.to_csv('growth.csv',index = False)
updated_growth_df.to_html('growth.html',index = False)
updated_growth_df.to_json('growth.json',orient = "records")

In [24]:
# adding lat long info 
countries_df = pd.read_csv("country_lan_lng.csv",engine='python')
merged = corona_df.merge(countries_df, left_on='Country,Other', right_on='name')
df_used = merged.drop(countries_df.columns[0:-2], axis=1)

df_used.rename(columns={"Country,Other":"Country","Serious,Critical": "CriticalCases"}, inplace = True)

#replace items using wild card and regex to have that applied
df_used["NewCases"].replace("\,","",regex = True, inplace = True)
df_used["NewCases"].replace("\+","", regex = True, inplace = True)



df_country_list = df_used["Country"].tolist()
df_used
#df_country_list

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,ActiveCases,CriticalCases,Tot Cases/1M pop,Longitude,Latitude
0,China,80783,29.0,3158.0,22.0,61493.0,16132,4492.0,56.1,103.819074,36.561765
1,Italy,10149,0.0,631.0,0.0,1004.0,8514,877.0,167.9,12.070013,42.796626
2,Iran,8042,0.0,291.0,0.0,2731.0,5020,0.0,95.7,54.274070,32.575033
3,S. Korea,7755,242.0,61.0,1.0,288.0,7406,54.0,151.3,127.839161,36.385240
4,France,1784,0.0,33.0,0.0,12.0,1739,86.0,27.3,-2.761729,42.173440
...,...,...,...,...,...,...,...,...,...,...,...
104,Jamaica,1,0.0,0.0,0.0,0.0,1,0.0,0.3,-77.314826,18.156949
105,Liechtenstein,1,0.0,0.0,0.0,0.0,1,0.0,0.0,9.535743,47.136658
106,Mongolia,1,0.0,0.0,0.0,0.0,1,0.0,0.3,103.052998,46.826815
107,Togo,1,0.0,0.0,0.0,0.0,1,0.0,0.1,0.962328,8.525314


In [25]:
df_used.to_csv('data.csv')

In [26]:
corona_json = df_used.to_json(orient='records')
df_used.to_json('data.json',orient='records')

In [27]:
corona_json

'[{"Country":"China","TotalCases":80783,"NewCases":29.0,"TotalDeaths":3158.0,"NewDeaths":22.0,"TotalRecovered":61493.0,"ActiveCases":16132,"CriticalCases":4492.0,"Tot\\u00a0Cases\\/1M pop":56.1,"Longitude":103.8190735,"Latitude":36.56176546},{"Country":"Italy","TotalCases":10149,"NewCases":0.0,"TotalDeaths":631.0,"NewDeaths":0.0,"TotalRecovered":1004.0,"ActiveCases":8514,"CriticalCases":877.0,"Tot\\u00a0Cases\\/1M pop":167.9,"Longitude":12.07001339,"Latitude":42.79662641},{"Country":"Iran","TotalCases":8042,"NewCases":0.0,"TotalDeaths":291.0,"NewDeaths":0.0,"TotalRecovered":2731.0,"ActiveCases":5020,"CriticalCases":0.0,"Tot\\u00a0Cases\\/1M pop":95.7,"Longitude":54.27407004,"Latitude":32.57503292},{"Country":"S. Korea","TotalCases":7755,"NewCases":242.0,"TotalDeaths":61.0,"NewDeaths":1.0,"TotalRecovered":288.0,"ActiveCases":7406,"CriticalCases":54.0,"Tot\\u00a0Cases\\/1M pop":151.3,"Longitude":127.8391609,"Latitude":36.38523983},{"Country":"France","TotalCases":1784,"NewCases":0.0,"Tot